# Introduction
[insert intro]

In [ ]:
%pip install pandas
%pip install matplotlib

In [ ]:
import math
import pandas as pd
import matplotlib.pyplot as plt

# Constants
g = 9.81                  # gravity [m/s^2]
rho = 1000                # density of water [kg/m^3]
mu = 0.001                # dynamic viscosity [Pa·s]
eps = 1.5e-6              # pipe roughness [m] (smooth plastic)
D = 0.00794               # pipe diameter [m]
A = math.pi * (D / 2) ** 2  # pipe cross-sectional area [m^2]

# Estimate tank area (cylinder with radius ~2.5cm)
tank_radius = 0.025      # [m]
A_t = math.pi * tank_radius ** 2  # tank cross-sectional area [m^2]

# Height values
h0 = 0.08    # initial height [m]
hf = 0.00    # final height [m]
sqrt_h_term = math.sqrt(h0) - math.sqrt(hf)

In [ ]:
# Compute drain time for given L and K
def compute_drain_time(L, K):
    f = 0.0
    v = 0.0
    Re = 0.0

    # TODO: Iterate to find f and v

    T = (2 * A_t / A) * math.sqrt((1 + f * L / D + K) / (2 * g)) * sqrt_h_term
    return round(T, 2), round(v, 3), int(Re), round(f, 5)

In [ ]:
cases = [
    {"label": "L = 20 cm", "L": 0.20, "K": 1.0},
    {"label": "L = 30 cm", "L": 0.30, "K": 1.0},
    {"label": "L = 40 cm", "L": 0.40, "K": 1.0},
    {"label": "L = 60 cm", "L": 0.60, "K": 1.0},
    {"label": "L = 40 cm + T-joint", "L": 0.40, "K": 2.0},
]


# Conclusion
[insert results and conclusion]